In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from evoman.environment import Environment
from controller_custom import player_controller
from multiprocessing import Pool
from scipy.stats import ttest_ind, mannwhitneyu
import pandas as pd
from deap import base, creator, tools, algorithms
import warnings

pygame 2.6.0 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

In [3]:
def run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group):
    """
    Runs Genetic Algorithm B with Crowding for preserving diversity.

    Parameters:
    - env: Evoman Environment instance.
    - npopulation: Population size.
    - gens: Number of generations.
    - mutation_rate: Mutation rate.
    - dom_u: Upper bound for gene values.
    - dom_l: Lower bound for gene values.
    - crowding_factor: Factor to determine the number of individuals to compete.

    Returns:
    - history_mean: List of mean fitness per generation.
    - history_max: List of max fitness per generation.
    - best_solution: Best solution found.
    """
    
    n_hidden_neurons = 10
    n_vars = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5

    # Run the simulation and return the fitness
    def simulate(x):
        env.player_controller.set(x, env.get_num_sensors())
        f, _, _, _ = env.play(pcont=x)
        return f

    # Evaluate the current population
    def evaluate(population):
        return np.array([simulate(individual) for individual in population])

    # Tournament Selection
    def tournament_selection(population, fitness, k=5):
        selected = []
        for _ in range(len(population)):
            contenders = np.random.choice(len(population), k, replace=False)
            winner = contenders[np.argmax(fitness[contenders])]
            selected.append(population[winner])
        return np.array(selected)

    # Uniform Crossover
    def crossover(parent1, parent2):
        mask = np.random.rand(n_vars) < 0.5
        child1 = np.where(mask, parent1, parent2)
        child2 = np.where(mask, parent2, parent1)
        return child1, child2

    # Gaussian Mutation
    def mutate(child):
        for i in range(n_vars):
            if np.random.rand() < mutation_rate:
                child[i] += np.random.normal(0, 0.1)
                child[i] = np.clip(child[i], dom_l, dom_u)
        return child


    # Initialize population
    population = np.random.uniform(dom_l, dom_u, (npopulation, n_vars))
    fitness = evaluate(population)

    # Record fitness over generations
    history_mean = []
    history_max = []
    history_std = []

    # Genetic Algorithm Loop
    for generation in range(1, gens + 1):
        # Selection
        selected = tournament_selection(population, fitness)

        # Crossover 
        for i in range(0, npopulation-1, 2):
            parent1, parent2 = selected[i], selected[i + 1]
            child1, child2 = crossover(parent1, parent2)
            
            # Mutation
            child1 = mutate(child1)
            child2 = mutate(child2)

            # Calculate distances (Euclidean) between parents and offspring
            d_p1_o1 = np.linalg.norm(parent1 - child1)
            d_p1_o2 = np.linalg.norm(parent1 - child2)
            d_p2_o1 = np.linalg.norm(parent2 - child1)
            d_p2_o2 = np.linalg.norm(parent2 - child2)

            # Calculate fitness for offspring
            fitness_child1 = simulate(child1)
            fitness_child2 = simulate(child2)

            # Crowding
            if d_p1_o1 + d_p2_o2 < d_p1_o2 + d_p2_o1:
                # Pair child1 with parent1 and child2 with parent2
                if fitness_child1 > fitness[i]:
                    population[i] = child1
                if fitness_child2 > fitness[i + 1]:
                    population[i + 1] = child2
            else:
                # Pair child1 with parent2 and child2 with parent1
                if fitness_child1 > fitness[i + 1]:
                    population[i + 1] = child1
                if fitness_child2 > fitness[i]:
                    population[i] = child2 

        # Recalculate fitness for the updated population
        fitness = evaluate(population)

        # Record statistics for the generation
        history_mean.append(np.mean(fitness))
        history_max.append(np.max(fitness))
        history_std.append(np.std(fitness))

        # Logging
        print(f'Generation {generation}: Best Fitness = {history_max[-1]:.4f}, Mean Fitness = {history_mean[-1]:.4f}, Std = {history_std[-1]:.4f}')

    # Get best solution
    best_idx = np.argmax(fitness)
    best_solution = population[best_idx]

    np.savetxt(os.path.join('results', f'best_solution_B_{enemy_group}.txt'), best_solution)

    return history_mean, history_max, best_solution


## Parameters

In [4]:
# Experiment Parameters

enemy_group_1 = [1, 4, 5, 8]
enemy_group_2 = [2, 3, 6, 7]
enemy_group_3 = np.arange(1,9)

num_runs = 2 			# Set to 10
npopulation = 200
gens = 10 				# Set to 30
mutation_rate = 0.05
dom_u, dom_l = 1, -1
elitism_count_a = 1
elitism_count_b = 2
 

# The number of hidden neurons for the player controller
n_hidden_neurons = 10

# Initialize data storage
results = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Generation': [],
    'Mean Fitness': [],
    'Max Fitness': [],
    'Best Solution': []
}

best_solutions = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Best Solution': []
}

# Run Algorithm B on Enemy Group 1

In [5]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [10]:
def objective(params, env, dom_u, dom_l, enemy_group='enemy_group_1'):
    params['npopulation'] = int(params['npopulation'])
    params['gens'] = int(params['gens'])
    
    # Pass dom_u, dom_l, and enemy_group inside params dictionary
    history_mean, history_max, solution = run_algorithm_b(env, **params, dom_u=dom_u, dom_l=dom_l, enemy_group=enemy_group)
    
    return {'loss': -history_max[-1], 'status': STATUS_OK}

def hyperopt_optimization(env, dom_u, dom_l, enemy_group='enemy_group_1', max_evals=5):
    space = {
        'npopulation': hp.quniform('npopulation', 100, 200, 10),
        'gens': hp.quniform('gens', 20, 30, 2),
        'mutation_rate': hp.uniform('mutation_rate', 0.01, 0.1),
    }

    trials = Trials()
    best = fmin(fn=lambda params: objective(params, env, dom_u, dom_l, enemy_group),
                space=space,
                algo=tpe.suggest,
                max_evals=max_evals,
                trials=trials)

    best['npopulation'] = int(best['npopulation'])
    best['gens'] = int(best['gens'])

    return best


In [11]:
experiment_name = f'optimization_enemy_EG1'
os.makedirs(experiment_name, exist_ok=True)

env = Environment(
    experiment_name=experiment_name,
    multiplemode="yes",
    enemies=enemy_group_1,
    playermode="ai",
    player_controller=player_controller(n_hidden_neurons),
    enemymode="static",
    level=2,
    speed="fastest",
    visuals=False)

best_params = hyperopt_optimization(env, dom_u, dom_l)

print(f"Best parameters for enemy: {best_params}")
print("--------------------")



MESSAGE: Pygame initialized for simulation.
Generation 1: Best Fitness = 32.0387, Mean Fitness = -1.7974, Std = 8.3247
Generation 2: Best Fitness = 32.0387, Mean Fitness = 2.8440, Std = 9.3303
Generation 3: Best Fitness = 32.0387, Mean Fitness = 7.1152, Std = 9.8341
Generation 4: Best Fitness = 35.8827, Mean Fitness = 11.4629, Std = 8.8797
Generation 5: Best Fitness = 44.9835, Mean Fitness = 15.0048, Std = 9.2741
Generation 6: Best Fitness = 56.4276, Mean Fitness = 18.4039, Std = 9.7352
Generation 7: Best Fitness = 56.4276, Mean Fitness = 21.6494, Std = 9.9018
Generation 8: Best Fitness = 56.4276, Mean Fitness = 24.6525, Std = 9.6180
Generation 9: Best Fitness = 64.0357, Mean Fitness = 26.9604, Std = 9.8423
Generation 10: Best Fitness = 64.0357, Mean Fitness = 28.3940, Std = 9.6545
Generation 11: Best Fitness = 69.6153, Mean Fitness = 30.5642, Std = 10.5051
Generation 12: Best Fitness = 69.6153, Mean Fitness = 32.3717, Std = 10.5759
Generation 13: Best Fitness = 69.6153, Mean Fitness 

## EA2 on EG1

In [9]:
print('Running Algorithm B for enemy group 3')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA2_EG1'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_1,
						playermode="ai",
						multiplemode="yes",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_1')

	for gen in range(gens):
		results['Algorithm'].append('B')
		results['Enemy Group'].append(1)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('B')
	best_solutions['Enemy Group'].append(1)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA2_EG1.csv'), index=False)

Running Algorithm B for enemy group 3
Run 1

MESSAGE: Pygame initialized for simulation.
Generation 0: Best Fitness = 21.8600, Mean Fitness = -1.2595, Std = 6.5164
Generation 1: Best Fitness = 24.8009, Mean Fitness = 4.1698, Std = 7.8408
Generation 2: Best Fitness = 29.6240, Mean Fitness = 9.4904, Std = 7.6730
Generation 3: Best Fitness = 29.6240, Mean Fitness = 14.2251, Std = 7.1071
Generation 4: Best Fitness = 31.2466, Mean Fitness = 19.1973, Std = 6.1319
Generation 5: Best Fitness = 31.2466, Mean Fitness = 21.8275, Std = 5.3545
Generation 6: Best Fitness = 32.4189, Mean Fitness = 24.9940, Std = 4.7176
Generation 7: Best Fitness = 32.4189, Mean Fitness = 26.7364, Std = 4.0863
Generation 8: Best Fitness = 33.9174, Mean Fitness = 28.2226, Std = 2.8616
Generation 9: Best Fitness = 33.9174, Mean Fitness = 29.0141, Std = 2.1100
Run 2

MESSAGE: Pygame initialized for simulation.
Generation 0: Best Fitness = 29.9875, Mean Fitness = 0.6659, Std = 8.4460
Generation 1: Best Fitness = 49.2106, 